## Concha

Concha calculcates the optimal number of things to make/order of perishables.
It uses machine learning, the day of the week, the weather, and the profit margin
of each product to optimally balance filling customer demand and reducing waste.

Open this notebook up in Google Colab to run it on Google's computers (it's free...and amazing).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ready4theCrush/concha/blob/master/notebooks/concha_examples.ipynb)

In [5]:
# Run this cell first to bring in the concha code
!pip -q install concha
from concha import Planner

In [6]:
# First, let's run a simulation for muffins.
# Let's say they're made in batches of 4, the marginal cost is
# $1.50 per muffin ($6 per batch), and they sell for $3.00 each.
sim_planner = Planner(
    planner_name="cafe_2",
    batch_size = 4,
    batch_cost= 6.0,
    unit_sale_price= 3.0
)

# We'll simulate 180 days of sales for 5 kinds of muffins
sim_planner.simulate_history(
    num_days=180,
    stockout_prob=0.5,
    demand_mean=100,
    demand_std=10,
    num_products = 5
) 

In [6]:
# We'll compare profits from two different ways to plan production.
# The first is to use the average for weekdays and weekends from past
# days to plan future production. The other uses machine learning to
# maximize profit (revenue - waste). 
sim_planner.grid_search(
    param_grid={"model": [
        "MeanWeekPart",
        "ProfitMaximizer"
    ]},
    true_demand='demand'
)

Evaluating: {'model': 'MeanWeekPart'}
Evaluating: {'model': 'ProfitMaximizer'}


,model,label,profit_avg,waste_pct,profit,val_loss,waste
0,MeanWeekPart,model=MeanWeekPart,65.606667,15.070632,"[[110.92, 107.92, 106.67, 109.33, 108.08, 44.5...",[[]],"[[3.64, 5.25, 5.56, 4.11, 4.53, 10.97, 16.28, ..."
1,ProfitMaximizer,model=ProfitMaximizer,87.110000,10.189812,"[[137.75, 127.67, 121.5, 139.83, 141.58, 57.0,...","[[1.389, 1.3094, 1.2381, 1.1785, 1.1238, 1.074...","[[5.58, 9.61, 4.61, 5.17, 7.08, 9.06, 9.86, 6...."


In [4]:
# To predict how much to make of each product, we
# just need to train the model for each product on the 
# transaction history first (which is simulated right now). 

sim_planner.train()
production = sim_planner.predict()

# The predictions are written to the forecast/forecast_production.csv file.
# Here's the first five rows of predictions.
production.head()

/home/thomas/concha_planners/cafe_2
No transaction csv files to import.
Training model for product: 1
Training model for product: 2
Training model for product: 3
Training model for product: 4
Training model for product: 5
Predicting production for product: 1
Predicting production for product: 2
Predicting production for product: 3
Predicting production for product: 4
Predicting production for product: 5


,date,product,day_of_week,tmin,tmax,prcp,snow,production
0,2020-10-09,1,Fri,54.0,78.0,False,False,48.0
1,2020-10-10,1,Sat,57.0,77.0,False,False,148.0
2,2020-10-11,1,Sun,57.0,80.0,False,False,100.0
3,2020-10-12,1,Mon,57.0,91.0,False,False,56.0
4,2020-10-13,1,Tue,57.0,95.0,False,False,32.0


In [ ]:
# Importing the transaction history

# From the POS you need a csv dump of the transactions with
# three columns (in this order): timestamp, item name, quantity.
sim_planner.import_transactions()
sim_planner.update_settings()

In [3]:
# Using the weather

# The weather history and forecasts come from NOAA. You'll
# need to get a NOAA api key from here: https://www.ncdc.noaa.gov/cdo-web/token
# You can set it in the planner with:
sim_planner.noaa_key("Yourkeyhere")

NOAA station set.


In [5]:
# To find a station near your business location you can
# pick one off the NOAA map. Go here: https://www.ncdc.noaa.gov/cdo-web/search
#  - For "Select a Dataset" pick "Daily Summaries"
#  - For "Enter a Search Term" put in your city/state name. Hit "SEARCH" and a map should come up.
#  - On the left side of the map, above the search results and below the "Search" box,
#    click on "More Search Options" and check the "Air Temperature" and "Precipitation"
#    boxes. Then click "SEARCH" again. When you find the nearest station, click on it
#    and copy the ID. It will look like "GHCND:USC00448084". You can set it with:
sim_planner.noaa_station("GHCND:USC00448084")